In [74]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
import os
import numpy as np
from shapely.geometry import box
from tqdm.notebook import tqdm

# 경고 무시 모듈
import warnings

# 경고를 무시
warnings.filterwarnings(action='ignore')

# 경고 보이기  
#warnings.filterwarnings(action='default')

In [75]:
li = os.listdir('./DATA/김해 건축물 정보/')
li

['김해건물_5179.cpg',
 '김해건물_5179.dbf',
 '김해건물_5179.prj',
 '김해건물_5179.qmd',
 '김해건물_5179.shp',
 '김해건물_5179.shx']

In [125]:
epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
epsg5179 = from_string("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs")

In [144]:
path = 'D:\김해시청 프로젝트\의료\김해시_의료지도_건물\의료기관_all'
path = path.split('\\')
path = '/'.join(path)

In [145]:
df_point = gpd.GeoDataFrame.from_file(path + '/의료기관_종합.shp', encoding='cp949', crs=epsg4326)

In [146]:
#df_point['geometry'] = df_point.apply(lambda row : Point([row['좌표정보(X'], row['좌표정보(Y']]), axis=1)

In [147]:
df_point['geometry'][562] = Point([df_point['좌표정보(X'][562], df_point['좌표정보(Y'][562]])

In [148]:
df_point = df_point.to_crs(epsg5179)

In [150]:
df_point.drop(['한방내과', '한방부인과', '한방소아과', '한방안·이', '한방신경정',
       '침구과', '한방재활의', '사상체질과', '한방응급과', '내과', '가정의학과', '정형외과', '마취통증의',
       '진단검사의', '재활의학과', '구강악안면', '소아치과', '구강내과', '예방치과', '치과', '치과보철과',
       '치과교정과', '치주과', '치과보존과', '영상치의학', '구강병리과', '구강악안_1', '통합치의학', '신경과',
       '외과', '신경외과', '산부인과', '소아청소년', '이비인후과', '비뇨의학과', '영상의학과', '응급의학과',
       '직업환경의', '정신건강의', '피부과', '흉부외과', '성형외과', '안과', '병리과', '결핵과', '예방의학과',
       '보건진료소', '보건지소', '보건소'],axis=1, inplace=True)

In [151]:
df = pd.DataFrame(columns=df_point.columns)

for n in tqdm(range(len(df_point))):
    
    # 병원별 진료과목 리스트
    j_li = df_point['진료과목_1'][n].split(',')
    
    # 1개 병원에 대해 진료과목별 포인트 생성
    for j in j_li:
        a = dict(df_point.loc[n])
        a['진료과목_1'] = j
        df = df.append(a, ignore_index=True)

  0%|          | 0/563 [00:00<?, ?it/s]

In [153]:
df_point = gpd.GeoDataFrame(df, geometry='geometry', crs = epsg5179)

In [154]:
%%time
df_polygon = gpd.GeoDataFrame.from_file('./DATA/김해 건축물 정보/김해건물_5179.shp', encoding='cp949')
df_polygon.crs = epsg5179

CPU times: total: 18.1 s
Wall time: 18.3 s


In [155]:
len(df_point)

4036

In [156]:
len(df_polygon)

53960

In [157]:
df_polygon['geo'] = df_polygon['geometry']

In [158]:
result = gpd.sjoin(df_point, df_polygon, how='left', op="intersects")

In [160]:
result['geometry'] = result['geo']

In [161]:
result.drop('geo', axis=1, inplace=True)

In [162]:
result

개방서비스      개방서비_1    개방자치단                       관리번호       인허가일자 인허가취소  \
0       병원  01_01_01_P  5350000  PHMA219975350029021200001  19970401.0  None   
1       병원  01_01_01_P  5350000  PHMA219975350029021200001  19970401.0  None   
2       병원  01_01_01_P  5350000  PHMA219975350029021200001  19970401.0  None   
3       병원  01_01_01_P  5350000  PHMA219975350029021200001  19970401.0  None   
4       병원  01_01_01_P  5350000  PHMA219975350029021200001  19970401.0  None   
...    ...         ...      ...                        ...         ...   ...   
4031    의원  01_01_02_P  5350000  PHMA120085350029041100003  20080129.0  None   
4032    의원  01_01_02_P  5350000                       None         NaN  None   
4033    의원  01_01_02_P  5350000                       None         NaN  None   
4034    의원  01_01_02_P  5350000                       None         NaN  None   
4035    의원  01_01_02_P  5350000                       None         NaN  None   

     영업상태구  영업상태명 상세영업상 상세영업_1  ...    A26        A27 A28    A29     A30  A31  \
0        1  영업/정상    13    영업중  ...  09100         병원   6  문교사회용  28.759  7.0   
1        1  영업/정상    13    영업중  ...  09100         병원   6  문교사회용  28.759  7.0   
2        1  영업/정상    13    영업중  ...  09100         병원   6  문교사회용  28.759  7.0   
3        1  영업/정상    13    영업중  ...  09100         병원   6  문교사회용  28.759  7.0   
4        1  영업/정상    13    영업중  ...  09100         병원   6  문교사회용  28.759  7.0   
...    ...    ...   ...    ...  ...    ...        ...  ..    ...     ...  ...   
4031     1  영업/정상    13    영업중  ...  04000  제2종근린생활시설   2    상업용  12.900  2.0   
4032     1  영업/정상    13    영업중  ...  10003         학원   6  문교사회용  24.000  6.0   
4033     1  영업/정상    13    영업중  ...  10003         학원   6  문교사회용  24.000  6.0   
4034     1  영업/정상    13    영업중  ...  10003         학원   6  문교사회용  24.000  6.0   
4035     1  영업/정상    13    영업중  ...  10003         학원   6  문교사회용  24.000  6.0   

      A32         A33         A34         A35  
0     2.0  1997-09-22  2001-02-16  2022-01-17  
1     2.0  1997-09-22  2001-02-16  2022-01-17  
2     2.0  1997-09-22  2001-02-16  2022-01-17  
3     2.0  1997-09-22  2001-02-16  2022-01-17  
4     2.0  1997-09-22  2001-02-16  2022-01-17  
...   ...         ...         ...         ...  
4031  0.0  1998-03-30  1998-06-11  2022-01-17  
4032  1.0        None  2003-01-27  2022-01-17  
4033  1.0        None  2003-01-27  2022-01-17  
4034  1.0        None  2003-01-27  2022-01-17  
4035  1.0        None  2003-01-27  2022-01-17  

[4036 rows x 81 columns]

In [163]:
result.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 4036 entries, 0 to 4035
Data columns (total 81 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   개방서비스        4036 non-null   object  
 1   개방서비_1       4036 non-null   object  
 2   개방자치단        4036 non-null   object  
 3   관리번호         4032 non-null   object  
 4   인허가일자        4032 non-null   float64 
 5   인허가취소        0 non-null      object  
 6   영업상태구        4036 non-null   object  
 7   영업상태명        4036 non-null   object  
 8   상세영업상        4036 non-null   object  
 9   상세영업_1       4036 non-null   object  
 10  폐업일자         0 non-null      object  
 11  휴업시작일        0 non-null      object  
 12  휴업종료일        0 non-null      object  
 13  재개업일자        0 non-null      object  
 14  소재지전화        4032 non-null   object  
 15  소재지면적        0 non-null      object  
 16  소재지우편        959 non-null    float64 
 17  소재지전체        3561 non-null   object  
 18  도로명전체        4036 no

In [164]:
#result_null = result[result['A29'].isnull()]
#result_null.to_file(f'{directory}김해시_의료기관_건물정보_null.shp',encoding='cp949',  driver='ESRI Shapefile')

In [165]:
# 폴더 생성
try:
    directory = './전처리/의료기관_건물정보/'
    if not os.path.exists(directory):
        os.makedirs(directory)
except OSError:
    print ('Error: Creating directory. ' +  directory)

result.to_file(f'{directory}김해시_의료기관_건물정보.shp',encoding='cp949',  driver='ESRI Shapefile')

In [166]:
result[result['A29'].isnull()].value_counts('사업장명')

사업장명
김정주정형외과의원         16
서종요양병원            13
뉴욕치과병원            12
하이클래스치과의원         11
진영서울치과의원          11
                  ..
최안과의원              1
서부건강지원센터           1
늘사랑가족의원            1
경희리틀병원             1
(의)동남의료재단 동남병원     1
Length: 66, dtype: int64